In [1]:

# Standard libraries
import collections
import csv
import io
import json
import re
import sys
import os


# Third-party libraries
import numpy as np
import pandas as pd
from scipy.stats import norm
# Add the parent directory to sys.path
sys.path.append(os.path.abspath(".."))  # Parent directory of 'soil_id'

# Now import modules as a package
from soil_id import config
from soil_id.db import (
    get_WISE30sec_data,
    get_WRB_descriptions,
    getSG_descriptions,
    load_model_output,
    save_model_output,
    save_rank_output,
    save_soilgrids_output,
)
from soil_id.services import (
    get_soilgrids_classification_data,
    get_soilgrids_property_data,
)
from soil_id.utils import (
    agg_data_layer,
    assign_max_distance_scores,
    calculate_location_score,
    compute_data_completeness,
    drop_cokey_horz,
    extract_values,
    extract_WISE_data,
    getCF_fromClass,
    getClay,
    getProfile,
    getSand,
    getTexture,
    gower_distances,
    pedon_color,
    sg_get_and_agg,
    silt_calc,
    max_comp_depth,
    convert_geometry_to_utm,
    calculate_distances_and_intersections,
)

from soil_id.color import(
    calculate_deltaE2000,
)

In [2]:
# global
# Standard libraries
import logging
import math
import re

# Third-party libraries
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
from numpy.linalg import cholesky
from osgeo import ogr
from rosetta import SoilData, rosetta
from scipy.interpolate import UnivariateSpline
from scipy.sparse import issparse
from scipy.stats import entropy, norm
from shapely.geometry import Point, box
from sklearn.impute import SimpleImputer
from sklearn.metrics import pairwise
from sklearn.utils import validation

'''
def calculate_distances_and_intersections(mu_geo, point):
    """
    Calculate distances and intersections of geometries to a point.

    Args:
        mu_geo (GeoDataFrame): GeoDataFrame containing mapunit geometries.
        point (Point): Shapely Point object for the location of interest.

    Returns:
        DataFrame: Contains mapunit keys, distances, and intersection flags.
    """

    # Ensure the point is wrapped in a GeoDataFrame and projected correctly
    point_utm, epsg_code = convert_geometry_to_utm(point)
    
    # Ensure the point is a GeoDataFrame (for compatibility)
    if not isinstance(point_utm, gpd.GeoDataFrame):
        point_utm = gpd.GeoDataFrame(geometry=[point_utm], crs=epsg_code)
    
    # Transform the GeoDataFrame to the same UTM CRS
    mu_geo_utm = mu_geo.to_crs(epsg_code)
    
    # Reset index for clean operations
    mu_geo_utm = mu_geo_utm.reset_index(drop=True)
    point_utm = point_utm.reset_index(drop=True)
    
    # Extract the single geometry for the point
    point_geometry = point_utm.geometry.iloc[0]
    
    # Calculate distances and intersections
    distances = mu_geo_utm["geometry"].distance(point_geometry)
    intersects = mu_geo_utm["geometry"].intersects(point_geometry)
    return pd.DataFrame(
        {"MUGLB_NEW": mu_geo_utm["MUGLB_NEW"], "dist_meters": distances, "pt_intersect": intersects}
    )


def convert_geometry_to_utm(geometry, src_crs="epsg:4326", target_crs=None):
    """
    Transforms a given geometry from its source coordinate reference system (CRS)
    to an appropriate Universal Transverse Mercator (UTM) CRS based on the geometry's
    centroid location.

    Parameters:
    - geometry (shapely.geometry.base.BaseGeometry): The geometry to transform, which
      could be any type of geometry, e.g., Point, Polygon.
    - src_crs (str, optional): The EPSG code of the source CRS of the geometry. Defaults
      to 'epsg:4326'.
    - target_crs (str, optional): The EPSG code of the target CRS to which the geometry
      should be transformed. If None, the appropriate UTM CRS based on the geometry's
      centroid will be calculated.

    Returns:
    - tuple: A tuple containing the transformed geometry and the EPSG code of the
      target UTM CRS. The transformed geometry is in the new CRS, and distances from
      this geometry can be accurately calculated in linear units (meters).

    Notes:
    - If the target_crs is not provided, the function calculates the correct UTM zone
      based on the longitude (from -180 to 180 degrees) and adjusts for the northern or
      southern hemisphere based on the latitude.
    - This function assumes the geometry is already in the specified source CRS and will
      convert it directly to the target CRS without additional CRS transformations.
    """
    # If geometry is not a GeoDataFrame, wrap it into one
    if isinstance(geometry, Point):
        geometry = gpd.GeoDataFrame(geometry=[geometry], crs=src_crs)
    elif isinstance(geometry, gpd.GeoSeries):
        geometry = geometry.to_frame(name='geometry')

    # Project to source CRS (ensure proper handling)
    geometry = geometry.to_crs(src_crs)

    # Calculate the centroid
    centroid = geometry.centroid.iloc[0]
    lon, lat = centroid.x, centroid.y

    # Determine the UTM zone dynamically
    utm_zone = int((lon + 180) / 6) + 1
    hemisphere = "north" if lat >= 0 else "south"
    epsg_code = f"326{utm_zone:02d}" if hemisphere == "north" else f"327{utm_zone:02d}"
    target_crs = f"EPSG:{epsg_code}"

    # Reproject geometry to the UTM CRS
    geometry_utm = geometry.to_crs(target_crs)

    return geometry_utm, target_crs



def get_WISE30sec_data_csv(csv_file_path, MUGLB_NEW_Select):
    """
    Retrieve WISE 30-second data from a CSV file based on selected MUGLB_NEW values.

    Parameters:
    - csv_file_path (str): Path to the CSV file.
    - MUGLB_NEW_Select (list): List of MUGLB_NEW values to filter.

    Returns:
    - pd.DataFrame: Filtered data.
    """
    try:
        # Read the CSV file into a DataFrame
        data = pd.read_csv(csv_file_path)
        
        # Filter the DataFrame based on the MUGLB_NEW values
        filtered_data = data[data["MUGLB_NEW"].isin(MUGLB_NEW_Select)]
        
        # Select specific columns
        selected_columns = [
            "MUGLB_NEW", "COMPID", "id", "MU_GLOBAL", "NEWSUID", "SCID", "PROP", "CLAF",
            "PRID", "Layer", "TopDep", "BotDep", "CFRAG", "SDTO", "STPC", "CLPC",
            "CECS", "PHAQ", "ELCO", "SU_name", "FAO_SYS"
        ]
        filtered_data = filtered_data[selected_columns]
        
        return filtered_data

    except FileNotFoundError as e:
        logging.error(f"File not found: {e}")
        return None
    except Exception as err:
        logging.error(f"An error occurred: {err}")
        return None


def get_WRB_descriptions_from_txt(WRB_Comp_List, txt_file_path):
    """
    Retrieve WRB descriptions from a text file based on the provided WRB component list.

    Parameters:
    - WRB_Comp_List (list): List of WRB components to filter.
    - txt_file_path (str): Path to the text file containing WRB descriptions.

    Returns:
    - pd.DataFrame: Filtered DataFrame with WRB descriptions.
    """
    try:
        # Load the text file using pandas
        data = pd.read_csv(txt_file_path, delimiter="\t")
        
        # Ensure the column names match the expected format
        required_columns = [
            "WRB_tax",
            "WRB_tax_en",
            "Description_en",
            "Management_en",
            "WRB_tax_es",
            "Description_es",
            "Management_es",
            "WRB_tax_ks",
            "Description_ks",
            "Management_ks",
            "WRB_tax_fr",
            "Description_fr",
            "Management_fr",
        ]
        if not all(col in data.columns for col in required_columns):
            raise ValueError("Text file is missing required columns.")
        
        # Filter the data to include only rows with WRB_tax in WRB_Comp_List
        filtered_data = data[data["WRB_tax"].isin(WRB_Comp_List)]
        
        return filtered_data
    except Exception as err:
        logging.error(f"Error while retrieving WRB descriptions: {err}")
        return None
'''

'\ndef calculate_distances_and_intersections(mu_geo, point):\n    """\n    Calculate distances and intersections of geometries to a point.\n\n    Args:\n        mu_geo (GeoDataFrame): GeoDataFrame containing mapunit geometries.\n        point (Point): Shapely Point object for the location of interest.\n\n    Returns:\n        DataFrame: Contains mapunit keys, distances, and intersection flags.\n    """\n\n    # Ensure the point is wrapped in a GeoDataFrame and projected correctly\n    point_utm, epsg_code = convert_geometry_to_utm(point)\n    \n    # Ensure the point is a GeoDataFrame (for compatibility)\n    if not isinstance(point_utm, gpd.GeoDataFrame):\n        point_utm = gpd.GeoDataFrame(geometry=[point_utm], crs=epsg_code)\n    \n    # Transform the GeoDataFrame to the same UTM CRS\n    mu_geo_utm = mu_geo.to_crs(epsg_code)\n    \n    # Reset index for clean operations\n    mu_geo_utm = mu_geo_utm.reset_index(drop=True)\n    point_utm = point_utm.reset_index(drop=True)\n    \n  

In [3]:
wise_data = extract_WISE_data(lon = -1.57, lat=8.95, file_path = '/mnt/c/LandPKS_API_SoilID-master/global/wise30sec_poly_simp_soil.gpkg', buffer_dist=10000)
print(wise_data)

Using UTM CRS: EPSG:32630


/mnt/c/Users/jmaynard/Documents/GitHub/soil-id-algorithm/soil_id/utils.py:1404: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = geometry.centroid.iloc[0]
/mnt/c/Users/jmaynard/Documents/GitHub/soil-id-algorithm/soil_id/utils.py:84: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  mu_id_dist["distance"] = mu_id_dist.groupby("MUGLB_NEW")["dist_meters"].transform(min)


    MUGLB_NEW  COMPID    id  MU_GLOBAL     NEWSUID  SCID  PROP CLAF   PRID  \
0        1530  108119  5549       1530  WD10001530     1    60  LXp  LXp/A   
1        1530  108119  5550       1530  WD10001530     1    60  LXp  LXp/A   
2        1530  108119  5551       1530  WD10001530     1    60  LXp  LXp/A   
3        1530  108119  5552       1530  WD10001530     1    60  LXp  LXp/A   
4        1530  108119  5553       1530  WD10001530     1    60  LXp  LXp/A   
5        1530  108119  5554       1530  WD10001530     1    60  LXp  LXp/A   
6        1530  108120  5555       1530  WD10001530     2    30  LVg  LVg/A   
7        1530  108120  5556       1530  WD10001530     2    30  LVg  LVg/A   
8        1530  108120  5557       1530  WD10001530     2    30  LVg  LVg/A   
9        1530  108120  5558       1530  WD10001530     2    30  LVg  LVg/A   
10       1530  108120  5559       1530  WD10001530     2    30  LVg  LVg/A   
11       1530  108120  5560       1530  WD10001530     2    30  

In [6]:
##################################################################################################
#                                 getSoilLocationBasedGlobal                                     #
##################################################################################################
def getSoilLocationBasedGlobal(lon, lat, plot_id):
    # Extract HWSD-WISE Data
    # Note: Need to convert HWSD shp to gpkg file
    wise_data = extract_WISE_data(
        lon,
        lat,
        # Temporarily change file path
        file_path = '/mnt/c/LandPKS_API_SoilID-master/global/wise30sec_poly_simp_soil.gpkg',
        #file_path=config.WISE_PATH,
        #layer_name=None,
        buffer_dist=10000,
    )

    # Component Data
    mucompdata_pd = wise_data[["MUGLB_NEW", "SU_name", "distance", "PROP", "COMPID", "FAO_SYS"]]
    mucompdata_pd.columns = ["mukey", "compname", "distance", "share", "cokey", "fss"]
    mucompdata_pd["distance"] = pd.to_numeric(mucompdata_pd["distance"])
    mucompdata_pd["share"] = pd.to_numeric(mucompdata_pd["share"])
    mucompdata_pd = mucompdata_pd.drop_duplicates().reset_index(drop=True)
 
    ##############################################################################################
    # Individual probability
    # Based on Fan et al 2018 EQ 1, the conditional probability for each component is calculated
    # by taking the sum of all occurances of a component in the home and adjacent mapunits and
    # dividing this by the sum of all map units and components. We have modified this approach
    # so that each instance of a component occurance is evaluated separately and assinged a
    # weight and the max distance score for each component group is assigned to all component
    # instances.
    ##############################################################################################
    ExpCoeff = -0.00036888  # Decays to 0.25 @ 10km
    loc_scores = []
    mucompdata_grouped = mucompdata_pd.groupby(["mukey", "cokey"], sort=False)

    for (mukey, cokey), group in mucompdata_grouped:
        loc_score = calculate_location_score(group, ExpCoeff)
        loc_scores.append({"cokey": cokey, "mukey": mukey, "distance_score": round(loc_score, 3)})

    loc_top_pd = pd.DataFrame(loc_scores)
    loc_top_comp_prob = loc_top_pd.groupby("cokey").distance_score.sum()
    loc_bot_prob_sum = loc_top_pd.distance_score.sum()
    cond_prob = (loc_top_comp_prob / loc_bot_prob_sum).reset_index(name="distance_score")

    mucompdata_pd = pd.merge(mucompdata_pd, cond_prob, on="cokey", how="left")
    mucompdata_pd = mucompdata_pd.sort_values("distance_score", ascending=False)
    
    mucompdata_pd = mucompdata_pd.reset_index(drop=True)
    mucompdata_pd["distance"] = mucompdata_pd["distance"].round(4)
    mucompdata_pd["Index"] = mucompdata_pd.index

    # Group by component name
    mucompdata_grouped = mucompdata_pd.groupby("compname", sort=False)

    # Take at most 12 groups
    mucompdata_comp_grps = [group for _, group in mucompdata_grouped][:12]

    # Assign max distance scores to all members within each group
    soilIDList_out = [assign_max_distance_scores(group) for group in mucompdata_comp_grps]

    mucompdata_pd = pd.concat(soilIDList_out).reset_index(drop=True)
    comp_key = mucompdata_pd["cokey"].tolist()

    # -----------------------------------------------------------------------------------------------------------------
    # Create horizon data table
    columns_to_select = [
        "COMPID",
        "TopDep",
        "BotDep",
        "id",
        "Layer",
        "SDTO",
        "STPC",
        "CLPC",
        "CFRAG",
        "CECS",
        "PHAQ",
        "ELCO",
        "PROP",
        "SU_name",
        "FAO_SYS",
    ]
    new_column_names = [
        "cokey",
        "hzdept_r",
        "hzdepb_r",
        "chkey",
        "hzname",
        "sandtotal_r",
        "silttotal_r",
        "claytotal_r",
        "total_frag_volume",
        "CEC",
        "pH",
        "EC",
        "comppct_r",
        "compname",
        "fss",
    ]

    muhorzdata_pd = wise_data[columns_to_select]
    muhorzdata_pd.columns = new_column_names
    muhorzdata_pd = muhorzdata_pd[muhorzdata_pd["cokey"].isin(comp_key)]
    muhorzdata_pd[["hzdept_r", "hzdepb_r"]] = (
        muhorzdata_pd[["hzdept_r", "hzdepb_r"]].fillna(0).astype(int)
    )
    muhorzdata_pd["texture"] = muhorzdata_pd.apply(getTexture, axis=1)
    muhorzdata_pd["texture"] = muhorzdata_pd["texture"].apply(
        lambda x: str(x) if isinstance(x, np.ndarray) else x
    )

    # Rank components and sort by rank and depth
    cokey_Index = {key: rank for rank, key in enumerate(comp_key)}
    muhorzdata_pd["Comp_Rank"] = muhorzdata_pd["cokey"].map(cokey_Index)
    muhorzdata_pd.sort_values(["Comp_Rank", "hzdept_r"], inplace=True)
    muhorzdata_pd.drop(columns="Comp_Rank", inplace=True)
    muhorzdata_pd = muhorzdata_pd.drop_duplicates().reset_index(drop=True)

    # Check for duplicate component instances
    hz_drop = drop_cokey_horz(muhorzdata_pd)
    if hz_drop is not None:
        muhorzdata_pd = muhorzdata_pd[~muhorzdata_pd.cokey.isin(hz_drop)]

    # Update comp_key
    comp_key = muhorzdata_pd["cokey"].unique().tolist()

    # Subset mucompdata_pd by new compname_key and add suffix to name if there are duplicates
    mucompdata_pd = mucompdata_pd.loc[mucompdata_pd['cokey'].isin(comp_key)].reset_index(drop=True)
    mucompdata_pd["compname_grp"] = mucompdata_pd["compname"]
    
    # Sort by 'distance_score' (descending) and 'distance' (ascending), then reset the index
    mucompdata_pd = mucompdata_pd.sort_values(['distance_score', 'distance'], ascending=[False, True]).reset_index(drop=True)

    # Add suffix to duplicate names
    name_counts = collections.Counter(mucompdata_pd["compname"])
    for name, count in name_counts.items():
        if count > 1:
            for suffix in range(1, count + 1):
                mucompdata_pd.loc[mucompdata_pd["compname"] == name, "compname"] = name + str(
                    suffix
                )

    # Add modified compname to muhorzdata
    muhorzdata_name = muhorzdata_pd[["cokey"]].merge(
        mucompdata_pd[["cokey", "compname"]], on="cokey"
    )
    muhorzdata_pd["compname"] = muhorzdata_name["compname"]

    # Group data by cokey for texture
    muhorzdata_group_cokey = list(muhorzdata_pd.groupby("cokey", sort=False))

    # Initialize lists for storing data
    getProfile_cokey = []
    c_bottom_depths = []
    clay_texture = []
    snd_lyrs = []
    cly_lyrs = []
    txt_lyrs = []
    hz_lyrs = []
    rf_lyrs = []
    cec_lyrs = []
    ph_lyrs = []
    ec_lyrs = []

    for group_key, group in muhorzdata_group_cokey:
        profile = (
            group.sort_values(by="hzdept_r")
            .drop_duplicates(keep="first")
            .reset_index(drop=True)
        )

        c_very_bottom = max_comp_depth(profile)

        sand_pct_intpl = getProfile(profile, "sandtotal_r")
        sand_pct_intpl.columns = ["c_sandpct_intpl", "c_sandpct_intpl_grp"]
        clay_pct_intpl = getProfile(profile, "claytotal_r")
        clay_pct_intpl.columns = ["c_claypct_intpl", "c_claypct_intpl_grp"]
        cf_pct_intpl = getProfile(profile, "total_frag_volume")
        cf_pct_intpl.columns = ["c_cfpct_intpl", "c_cfpct_intpl_grp"]
        cec_intpl = getProfile(profile, "CEC")
        cec_intpl.columns = ["c_cec_intpl"]
        ph_intpl = getProfile(profile, "pH")
        ph_intpl.columns = ["c_ph_intpl"]
        ec_intpl = getProfile(profile, "EC")
        ec_intpl.columns = ["c_ec_intpl"]

        combined_data = pd.concat(
            [
                sand_pct_intpl[["c_sandpct_intpl_grp"]],  # DataFrame
                clay_pct_intpl[["c_claypct_intpl_grp"]],  # DataFrame
                cf_pct_intpl[["c_cfpct_intpl_grp"]],     # DataFrame
                pd.DataFrame(profile.compname.unique(), columns=["compname"]),  # Convert to DataFrame
                pd.DataFrame(profile.cokey.unique(), columns=["cokey"]),        # Convert to DataFrame
                pd.DataFrame(profile.comppct_r.unique(), columns=["comppct"]),  # Convert to DataFrame
            ],
            axis=1,
        )

        combined_data.columns = [
            "sandpct_intpl",
            "claypct_intpl",
            "rfv_intpl",
            "compname",
            "cokey",
            "comppct",
        ]

        c_bottom_temp = pd.DataFrame(
            {
                "cokey": [combined_data["cokey"].iloc[0]],
                "compname": [combined_data["compname"].iloc[0]],
                "c_very_bottom": [int(c_very_bottom)],
            }
        )

        snd_d, hz_depb = agg_data_layer(sand_pct_intpl.c_sandpct_intpl, bottom=c_bottom_temp["c_very_bottom"].iloc[0], depth=True)
        cly_d = agg_data_layer(clay_pct_intpl.c_claypct_intpl, bottom=c_bottom_temp["c_very_bottom"].iloc[0], depth=False)
        txt_d = [
            getTexture(row=None, sand=s, silt=(100 - (s + c)), clay=c) for s, c in zip(snd_d, cly_d)
        ]
        txt_d = pd.Series(txt_d, index=snd_d.index)

        rf_d = agg_data_layer(cf_pct_intpl.c_cfpct_intpl, bottom=c_bottom_temp["c_very_bottom"].iloc[0], depth=False)
        cec_d = agg_data_layer(cec_intpl.c_cec_intpl, bottom=c_bottom_temp["c_very_bottom"].iloc[0], depth=False)
        ph_d = agg_data_layer(ph_intpl.c_ph_intpl, bottom=c_bottom_temp["c_very_bottom"].iloc[0], depth=False)
        ec_d = agg_data_layer(ec_intpl.c_ec_intpl, bottom=c_bottom_temp["c_very_bottom"].iloc[0], depth=False)

        # Fill NaN values and append to lists
        for data_list, data in zip(
            [snd_lyrs, cly_lyrs, txt_lyrs, rf_lyrs, cec_lyrs, ph_lyrs, ec_lyrs, hz_lyrs],
            [snd_d, cly_d, txt_d, rf_d, cec_d, ph_d, ec_d, hz_depb],
        ):
            data_list.append(dict(data.fillna("")))

        c_bottom_depths.append(c_bottom_temp)
        getProfile_cokey.append(combined_data)

        comp_texture_list = [x for x in profile.texture.str.lower() if x]
        clay_val = "Yes" if any("clay" in string for string in comp_texture_list) else "No"
        clay_texture_temp = pd.DataFrame(
            {"compname": [combined_data["compname"].iloc[0]], "clay": [clay_val]}
        )
        clay_texture.append(clay_texture_temp)

    # Concatenate lists to form DataFrames
    c_bottom_depths = pd.concat(c_bottom_depths, axis=0)
    clay_texture = pd.concat(clay_texture, axis=0)
    
    # Subset mucompdata and muhorzdata DataFrames
    mucompdata_pd = mucompdata_pd[mucompdata_pd["cokey"].isin(c_bottom_depths.cokey)]
    muhorzdata_pd = muhorzdata_pd[muhorzdata_pd["cokey"].isin(c_bottom_depths.cokey)]

    # Merge c_bottom_depth and clay_texture with mucompdata
    mucompdata_pd = pd.merge(
        mucompdata_pd, c_bottom_depths[["compname", "c_very_bottom"]], on="compname", how="left"
    )
    mucompdata_pd = pd.merge(mucompdata_pd, clay_texture, on="compname", how="left")

    # Create index for component instance display
    mucompdata_comp_grps_list = []
    mucompdata_comp_grps = [group for _, group in mucompdata_pd.groupby("compname_grp", sort=False)]

    for group in mucompdata_comp_grps:
        group = group.sort_values("distance").reset_index(drop=True)
        soilID_rank = [True if idx == 0 else False for idx in range(len(group))]

        group["soilID_rank"] = soilID_rank
        group["min_dist"] = group.distance.iloc[0]

        mucompdata_comp_grps_list.append(group)

    mucompdata_pd = pd.concat(mucompdata_comp_grps_list).reset_index(drop=True)

    # --------------------------------------------------------------------------------------------------------------------------
    # SoilIDList output

    # ------------------------------------------------------------
    # Format output data
    soilIDRank_output = [
        group[["compname", "sandpct_intpl", "claypct_intpl", "rfv_intpl"]]
        for group in getProfile_cokey
    ]
    soilIDRank_output_pd = pd.concat(soilIDRank_output, axis=0).reset_index(drop=True)

    mucompdata_cond_prob = mucompdata_pd.sort_values(
        "distance_score", ascending=False
    ).reset_index(drop=True)

    # Determine rank location
    rank_id = 1
    Rank_Loc = []
    for rank in mucompdata_cond_prob["soilID_rank"]:
        Rank_Loc.append(str(rank_id) if rank else "Not Displayed")
        rank_id += rank  # Increase rank_id only if rank is True
    mucompdata_cond_prob["Rank_Loc"] = Rank_Loc

    # --------------------------------------------------------------------------------------------------------------------------
    # API output: Code outputs HWSD data without any alteration (i.e., texture class averaging)

    # Handle NaN values
    mucompdata_cond_prob.replace([np.nan, "nan", "None", [None]], "", inplace=True)

    # Sort mucompdata_cond_prob by soilID_rank and distance_score
    mucompdata_cond_prob = mucompdata_cond_prob.sort_values(
        ["soilID_rank", "distance_score"], ascending=[False, False]
    )
    mucomp_index = mucompdata_cond_prob.index
    
    # Extract lists for constructing ID dictionary
    siteName = mucompdata_cond_prob["compname"].apply(lambda x: x.capitalize()).tolist()
    compName = mucompdata_cond_prob["compname_grp"].apply(lambda x: x.capitalize()).tolist()
    score = mucompdata_cond_prob["distance_score"].round(3).tolist()
    rank_loc = mucompdata_cond_prob["Rank_Loc"].tolist()
    model_version = 3

    # Step 3: Construct ID list directly from the sorted and cleaned DataFrame
    ID = []
    for i, idx in enumerate(mucompdata_cond_prob.index):
        idT = {
            "name": siteName[i],
            "component": compName[i],
            "score_loc": score[i],
            "rank_loc": rank_loc[i],
        }
        ID.append(idT)

    # Merge component descriptions
    WRB_Comp_Desc = get_WRB_descriptions(
        mucompdata_cond_prob["compname_grp"].drop_duplicates().tolist()
    )

    mucompdata_cond_prob = pd.merge(
        mucompdata_cond_prob, WRB_Comp_Desc, left_on="compname_grp", right_on="WRB_tax", how="left"
    )

    # Extract site information
    Site = [
        {
            "siteData": {
                "mapunitID": row.mukey,
                "componentID": row.cokey,
                "fao": row.fss,
                "share": row.share,
                "distance": round(row.distance, 3),
                "minCompDistance": row.min_dist,
                "soilDepth": row.c_very_bottom,
            },
            "siteDescription": {
                key: row[key]
                for key in [
                    "Description_en",
                    "Management_en",
                    "Description_es",
                    "Management_es",
                    "Description_ks",
                    "Management_ks",
                    "Description_fr",
                    "Management_fr",
                ]
            },
        }
        for _, row in mucompdata_cond_prob.iterrows()
    ]

    # Reorder lists based on mucomp_index
    hz_lyrs = [hz_lyrs[i] for i in mucomp_index]
    snd_lyrs = [snd_lyrs[i] for i in mucomp_index]
    cly_lyrs = [cly_lyrs[i] for i in mucomp_index]
    txt_lyrs = [txt_lyrs[i] for i in mucomp_index]
    rf_lyrs = [rf_lyrs[i] for i in mucomp_index]
    cec_lyrs = [cec_lyrs[i] for i in mucomp_index]
    ph_lyrs = [ph_lyrs[i] for i in mucomp_index]
    ec_lyrs = [ec_lyrs[i] for i in mucomp_index]

    output_SoilList = [
        dict(
            zip(
                [
                    "id",
                    "site",
                    "bottom_depth",
                    "sand",
                    "clay",
                    "texture",
                    "rock_fragments",
                    "cec",
                    "ph",
                    "ec",
                ],
                item,
            )
        )
        for item in zip(
            ID, Site, hz_lyrs, snd_lyrs, cly_lyrs, txt_lyrs, rf_lyrs, cec_lyrs, ph_lyrs, ec_lyrs
        )
    ]

    def convert_to_serializable(obj):
        if isinstance(obj, dict):
            return {k: convert_to_serializable(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_to_serializable(v) for v in obj]
        elif isinstance(obj, np.integer):  # Convert NumPy integers to Python int
            return int(obj)
        elif isinstance(obj, np.floating):  # Convert NumPy floats to Python float
            return float(obj)
        elif isinstance(obj, np.ndarray):  # Convert NumPy arrays to lists
            return obj.tolist()
        else:
            return obj
        
    output_SoilList_cleaned = convert_to_serializable(output_SoilList)

    # Save data
    if plot_id is None:
        soilIDRank_output_pd.to_csv(config.SOIL_ID_RANK_PATH, index=None, header=True)
        mucompdata_cond_prob.to_csv(config.SOIL_ID_PROB_PATH, index=None, header=True)
    else:
        save_model_output(
            plot_id,
            model_version,
            json.dumps(
                {
                    "metadata": {
                        "location": "global",
                        "model": "v2",
                        "unit_measure": {
                            "distance": "m",
                            "depth": "cm",
                            "cec": "cmol(c)/kg",
                            "clay": "%",
                            "rock_fragments": "cm3/100cm3",
                            "sand": "%",
                            "ec": "ds/m",
                        },
                    },
                    "soilList": output_SoilList_cleaned,
                }
            ),
            soilIDRank_output_pd.to_csv(index=None, header=True),
            mucompdata_cond_prob.to_csv(index=None, header=True),
        )

    # Return the JSON output
    return {
        "metadata": {
            "location": "global",
            "model": "v2",
            "unit_measure": {
                "distance": "m",
                "depth": "cm",
                "cec": "cmol(c)/kg",
                "clay": "%",
                "rock_fragments": "cm3/100cm3",
                "sand": "%",
                "ec": "ds/m",
            },
        },
        "soilList": output_SoilList_cleaned,
    }


In [7]:
global_return = getSoilLocationBasedGlobal(lon = -1.57, lat=8.95, plot_id=1)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(global_return)

Using UTM CRS: EPSG:32630


/mnt/c/Users/jmaynard/Documents/GitHub/soil-id-algorithm/soil_id/utils.py:1404: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = geometry.centroid.iloc[0]
/mnt/c/Users/jmaynard/Documents/GitHub/soil-id-algorithm/soil_id/utils.py:84: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  mu_id_dist["distance"] = mu_id_dist.groupby("MUGLB_NEW")["dist_meters"].transform(min)
/tmp/ipykernel_256532/971167363.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

{'metadata': {'location': 'global', 'model': 'v2', 'unit_measure': {'distance': 'm', 'depth': 'cm', 'cec': 'cmol(c)/kg', 'clay': '%', 'rock_fragments': 'cm3/100cm3', 'sand': '%', 'ec': 'ds/m'}}, 'soilList': [{'id': {'name': 'Ferric lixisols', 'component': 'Ferric lixisols', 'score_loc': 0.64, 'rank_loc': '1'}, 'site': {'siteData': {'mapunitID': 1453, 'componentID': 107895, 'fao': 'FAO90', 'share': 80, 'distance': 0.0, 'minCompDistance': 0.0, 'soilDepth': 120}, 'siteDescription': {'Description_en': 'Ferric Lixisols form in warm climates with relatively clayey subsoils dominated by kaolinite and iron oxides, but relatively high base saturation. <br> Ferric Lixisols have soft and cemented iron concentrations in subsoils, and poorly developed structure between iron concentrations which can be susceptible to compaction. ', 'Management_en': 'These soils are heavily weathered soils, which means that fertilization (and possibly lime) will be needed for production. <br> Although they do have ka

In [ ]:
print(global_return)